# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

#### GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### GroupBy in MongoDB
```
db.BusinessProcess.aggregate({
    "$group": {
        _id: {
            status: "$status",
            type: "$type"
        },
        count: {
            $sum: 1
        }
    }
   })
```


In [1]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.555370,1.174036
1,a,two,0.870194,0.220326
2,b,one,0.852130,-0.149336
3,b,two,0.383013,-0.020943
4,a,one,-0.587658,-1.751614


In [2]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [3]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one  0.555370  1.174036
1    a  two  0.870194  0.220326
4    a  one -0.587658 -1.751614
name: b 
   key1 key2     data1     data2
2    b  one  0.852130 -0.149336
3    b  two  0.383013 -0.020943


In [4]:
#access group table
grouped.get_group('b')

,key1,key2,data1,data2
2,b,one,0.852130,-0.149336
3,b,two,0.383013,-0.020943


In [5]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [6]:
#get number of group entries by columns
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


#### Think of  grouped DataFrames as 3d objects:

In [7]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0    1.174036
1    0.220326
4   -1.751614
Name: data2, dtype: float64

In [8]:
grouped.get_group('a')['data2']

0    1.174036
1    0.220326
4   -1.751614
Name: data2, dtype: float64

### Group by external keys

In [9]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005    0.602859
2006    0.132236
Name: data1, dtype: float64

### Group by functions

In [10]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    4    a  one -0.587658 -1.751614
     0    a  one  0.555370  1.174036
     1    a  two  0.870194  0.220326
b    3    b  two  0.383013 -0.020943
     2    b  one  0.852130 -0.149336

### Group-wise aggregation (apply)
<img SRC="IMG/groupby.jpg">

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [ ]:
#aggregate over the groups
grouped.count()

In [11]:
grouped.sum()

,data1,data2
key1,,
a,0.837907,-0.357251
b,1.235143,-0.170278


#### Custom Aggregation Functions

In [12]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.457852,2.925650
b,0.469117,0.128393


#### Multiple aggregations

In [13]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        1.457852  0.279302  0.555370     2.925650 -0.119084  0.220326
b        0.469117  0.617572  0.617572     0.128393 -0.085139 -0.085139

#### Suppressing the Group Keys 

In [14]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    0    a  one  0.555370  1.174036
     1    a  two  0.870194  0.220326
b    3    b  two  0.383013 -0.020943
     2    b  one  0.852130 -0.149336

In [15]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

,key1,key2,data1,data2
0,a,one,0.555370,1.174036
1,a,two,0.870194,0.220326
3,b,two,0.383013,-0.020943
2,b,one,0.852130,-0.149336


## More  Exercises in the Lab session... 